# Temperatura de brillo ASTER L1T google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap="ESRI")

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccion de la imagen

In [6]:
# Seleccionar una imagen
ASTER_img = ee.Image("ASTER/AST_L1T_003/20070730150418")\
              .select(["B10","B11","B12","B13","B14"])\
              .clip(zona_ee)

In [7]:
# Nombre de las bandas
print(ASTER_img.bandNames().getInfo())

['B10', 'B11', 'B12', 'B13', 'B14']


In [9]:
# Proyeccion se encuentra zona y resolucion espacial 90m B11
ASTER_img.projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32619',
 'transform': [90, 0, 333045, 0, -90, -1388385]}

In [10]:
# Reproyeccion segun zona
ASTER_img_utm = ASTER_img.reproject(crs="EPSG:32719", scale = 90)

In [11]:
# Consultar proyeccion
ASTER_img_utm.projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [90, 0, 0, 0, -90, 0]}

## Seleccion de las bandas

In [12]:
# Seleccionar las bandas
B10 = ASTER_img_utm.select("B10")
B11 = ASTER_img_utm.select("B11")
B12 = ASTER_img_utm.select("B12")
B13 = ASTER_img_utm.select("B13")
B14 = ASTER_img_utm.select("B14")

## Convertir a Radiancia sensor

In [14]:
# Convertir a Radiancia del sensor
# Del 2008 adelante no hay bandas SWIR 05 al 09
gain_B10 = ASTER_img_utm.get('GAIN_COEFFICIENT_B10').getInfo()
gain_B11 = ASTER_img_utm.get('GAIN_COEFFICIENT_B11').getInfo()
gain_B12 = ASTER_img_utm.get('GAIN_COEFFICIENT_B12').getInfo()
gain_B13 = ASTER_img_utm.get('GAIN_COEFFICIENT_B13').getInfo()
gain_B14 = ASTER_img_utm.get('GAIN_COEFFICIENT_B14').getInfo()

0.006822

In [15]:
# Determinar radiancia
L10 = B10.subtract(1).multiply(gain_B10)
L11 = B11.subtract(1).multiply(gain_B11)
L12 = B12.subtract(1).multiply(gain_B12)
L13 = B13.subtract(1).multiply(gain_B13)
L14 = B14.subtract(1).multiply(gain_B14)


## Determinar temperatura brillo ASTER L1T

In [16]:
# K imagen ASTER L1T
K1_B10 = ee.Image(3024)
K2_B10 = ee.Image(1733)
K1_B11 = ee.Image(2460)
K2_B11 = ee.Image(1663)
K1_B12 = ee.Image(1909)
K2_B12 = ee.Image(1581)
K1_B13 = ee.Image(890)
K2_B13 = ee.Image(1357)
K1_B14 = ee.Image(646.4)
K2_B14 = ee.Image(1273)

In [17]:
# Determinacion Temperatura de brillo Celsius
TB_B10 = K2_B10.divide(ee.Image.log(K1_B10.divide(L10).add(1))).subtract(273.15)
TB_B11 = K2_B11.divide(ee.Image.log(K1_B11.divide(L11).add(1))).subtract(273.15)
TB_B12 = K2_B12.divide(ee.Image.log(K1_B12.divide(L12).add(1))).subtract(273.15)
TB_B13 = K2_B13.divide(ee.Image.log(K1_B13.divide(L13).add(1))).subtract(273.15)
TB_B14 = K2_B14.divide(ee.Image.log(K1_B14.divide(L14).add(1))).subtract(273.15)

## Renombrar nombre bandas

In [18]:
# Composicion de bandas TB
TB_ASTER = TB_B10.addBands([TB_B11,TB_B12,TB_B13,TB_B14])

In [21]:
# Consulta nombre bandas
TB_ASTER.bandNames().getInfo()

['TB10', 'TB11', 'TB12', 'TB13', 'TB14']

In [20]:
# Renombrar bandas
TB_ASTER = TB_ASTER.rename("TB10","TB11","TB12","TB13","TB14")

In [22]:
# Crear su simbologia
viz_TB = {
    'min': 10,
    'max': 23,
    'palette': ['1021ef','e4ff0b','ff1c05']
}

In [23]:
# Visualizar 
Map.addLayer(TB_ASTER.select("TB10"), viz_TB,name = "TB B10 Celsi")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Exportar Imagen calibrado

In [24]:
import os

In [25]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [26]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [27]:
# Exportar en google drive
geemap.ee_export_image_to_drive(TB_ASTER, description='TB_ASTER_clip_B10_11_12_13_14', folder='GEE_Python', region=geometria, scale=90)

Exporting TB_ASTER_clip_B10_11_12_13_14 ...


In [ ]:
# Exportar en google drive
geemap.ee_export_image_to_drive(TB_ASTER.select("TB10"), description='TB_ASTER_clip_B10', folder='GEE_Python', region=geometria, scale=90)